## Functions

In [109]:
def islendingur():
    bakgrunnsuppl = input_bakgrunnsuppl()
    tekjur_e_skatt, stadgreidsla, lifeyrisgreidsla, sereignarlifeyrisgr = stadgreidsla_func(bakgrunnsuppl)
    husnaedisstudningur, husn_eignaskerding, husn_tekjuskerding =\
        husnaedisstudningur_func(bakgrunnsuppl)
    if hjuskaparstada != 1:
        barnabaetur = barnabaetur_func(bakgrunnsuppl)
    else: barnabaetur = 0
    
    if bakgrunnsuppl['tekjur'][0] >= 1678001/12:
        utvarpsgjald = 16800/12
    else: utvarpsgjald = 0
    
    netto = tekjur_e_skatt + husnaedisstudningur + barnabaetur - utvarpsgjald
    
    nidurstodur = {'tekjur': tekjur,\
                   'tekjur_e_skatt': tekjur_e_skatt,\
                   'stadgreidsla': stadgreidsla,\
                   'lifeyrisgreidsla': (lifeyrisgreidsla+sereignarlifeyrisgr),\
                   'husnaedisstudningur': husnaedisstudningur,\
                   'barnabaetur': barnabaetur,\
                   'utvarpsgjald': utvarpsgjald,\
                   'pjeng_i_vasa': netto}
    
    return nidurstodur

In [86]:
def input_bakgrunnsuppl():
    # HJÚSKAPARSTAÐA
    # =======================================================
    hjuskaparstada = 4
    while hjuskaparstada >3 or hjuskaparstada <1:
        hjuskaparstada = int(raw_input('1: einhleypur, 2: einst. foreldri, 3: giftur/sambud '))
    
    # BÚSETUFORM
    # =======================================================
    print "Hver er husnæðisstaða þín? Býrðu í eigin húsnæði, leiguhúsnæði/búsetu eða hvorugt?"
    husn = 4
    while husn >3 or husn <1:
        husn = int(raw_input('1: eigin, 2: leigu, 3: hvorugt '))
    
    # TEKJUR OG IÐGJALD
    # =======================================================
    laun_a_manudi = int(raw_input("Laun fyrir skatt: "))
    fjarmagnstekjur = int(raw_input("Fjarmagnstekjur: "))
    
    print "	Iðgjald í lífeyrissjóð?"
    idgjald = 3
    while idgjald >2 or idgjald <1:
        idgjald = int(raw_input('1: 0%, 2: 4% '))
    if idgjald == 1:
        idgjald = 0.0
    elif idgjald == 2:
        idgjald = 0.04
            
    print "	Iðgjald í séreign?"
    sereignaridgjald = 5
    while sereignaridgjald >4 or sereignaridgjald <0:
        sereignaridgjald = int(raw_input('0: 0%, 1: 1%, 2: 2%, 3: 3%, 4: 4% '))
    sereignaridgjald = sereignaridgjald / 100.0
    
    if hjuskaparstada == 3:
        laun_maka = int(raw_input("Laun maka fyrir skatt: "))
        fjarmagnstekjur_maka = int(raw_input("Fjarmagnstekjur maka: "))
    else:
        laun_maka = 0
        fjarmagnstekjur_maka = 0
    tekju_uppl = [laun_a_manudi, fjarmagnstekjur, laun_maka, fjarmagnstekjur_maka]
    
    # EIGNIR
    # =======================================================
    if hjuskaparstada == 3:
        print "Allar eignir (samanlagðar eignir hjóna) \
        að frádregnum öllum skuldum. Með eignum skal telja hlutabréf, innstæður og verðbréf: "
    else:
        print "Allar eignir að frádregnum öllum skuldum. Með eignum skal telja hlutabréf, innstæður og verðbréf: "
    eignir = int(raw_input())
    
    # HÚSNÆÐISBREYTUR
    # =======================================================
    if husn == 1:   
        print 'Eftirstöðvar í árslok af lánum sem tekin hafa verið til öflunar íbúðarhúsnæðis til eigin nota: '
        eftirstodvar = int(raw_input())
        print "Vextir og verðbætur af íbúðarlánum á ársgrundvelli, þ.m.t. dráttarvextir og lántökukostnaður: "
        vaxtagjold = int(raw_input())
    else:
        eftirstodvar = 0
        vaxtagjold = 0
    if husn == 2:
        fjoldi_heimilismanna = int(raw_input("fjoldi heimilismanna ad ther medtoldum: "))
        if fjoldi_heimilismanna > 4:
            fjoldi_heimilismanna = 4
        if fjoldi_heimilismanna == 1:
            heimilistekjur = (tekju_uppl[0] + tekju_uppl[1])
            heimiliseignir = eignir
        elif fjoldi_heimilismanna == 2 and hjuskaparstada == 3:
            heimilistekjur = sum(tekju_uppl)
            heimiliseignir = eignir
        else:
            heimilistekjur = int(raw_input('samanlagdar manadartekjur allra heimilismanna: '))
            heimiliseignir = int(raw_input('samanlagdar eignir allra heimilismanna: '))
        husnaediskostnadur = int(raw_input('manadarlegur husnaediskostnadur: '))
    else: 
        fjoldi_heimilismanna = 0
        heimilistekjur = 0
        heimiliseignir = 0
        husnaediskostnadur = 0
        
    # BÖRN
    # =======================================================
    if hjuskaparstada != 1:    
        print "Fjöldi barna á heimli: "
        fj_barna = int(raw_input())
        if fj_barna != 0:
            print " - þar af yngri en 7 ára: "
            fj_barna_undir_7 = fj_barna + 1
            while fj_barna_undir_7 > fj_barna:
                fj_barna_undir_7 = int(raw_input())
        elif fj_barna == 0:
            fj_barna_undir_7 = 0
        if fj_barna > 3:
            fj_barna = 3
    else:
        fj_barna = 0
        fj_barna_undir_7 = 0
    
    bakgrunnsuppl = {'hjuskaparstada': hjuskaparstada,\
                    'busetuform': husn,\
                    'tekjur': tekju_uppl,\
                    'eignir': eignir,\
                    'husnaedislan': eftirstodvar,\
                    'vaxtagjold': vaxtagjold,\
                    'husnaediskostnadur': husnaediskostnadur,\
                    'fjoldi_heimilismanna': fjoldi_heimilismanna,\
                    'heimilistekjur': heimilistekjur,\
                    'heimiliseignir': heimiliseignir,\
                    'fjoldi_barna': fj_barna,\
                    'fjoldi_barna_undir_7': fj_barna_undir_7,\
                    'idgjald': idgjald,\
                    'sereignaridgjald': sereignaridgjald
                    }
    
    return bakgrunnsuppl

In [99]:
def stadgreidsla_func(info):
    
    skattthrep = 834707
    skatthlutfall_nedra = 0.3694
    skatthlutfall_efra = 0.4624
    personuafsl = 52907
    
    lifeyrir = (info['tekjur'][0] * info['idgjald'])
    serlifeyrir = (info['tekjur'][0] * info['sereignaridgjald'])
    
    stofn = info['tekjur'][0] - lifeyrir - serlifeyrir
    
    if stofn <= skattthrep:
        skattstofn_1 = stofn
        skattstofn_2 = 0
    else:
        skattstofn_1 = skattthrep
        skattstofn_2 = stofn - skattthrep
        
    skattur_nedra_threp = (skattstofn_1*skatthlutfall_nedra)
    skattur_efra_threp = (skattstofn_2*skatthlutfall_efra)
    
    reiknud_stadgreidsla = skattur_nedra_threp + skattur_efra_threp
    stadgreidsla_alls = reiknud_stadgreidsla - personuafsl
    
    laun_e_skatt = stofn - stadgreidsla_alls
    
    return laun_e_skatt, stadgreidsla_alls, lifeyrir, serlifeyrir        

In [101]:
def husnaedisstudningur_func(info):

    if info['busetuform'] == 1:      
        studningur = vaxtabaetur_func(info)
        
    elif info['busetuform'] == 2:
        studningur = husnaedisbaetur_func(info)
        
    else:
        studningur = 0
    
    return studningur

In [73]:
def vaxtabaetur_func(info):
    
    hamark_vaxtagjalda = [800000, 1000000, 1200000] #fyrir: [einstakling, einstætt foreldri, hjón]
    hamark_bota = [400000, 500000, 600000] #fyrir: [einstakling, einstætt foreldri, hjón]
    tekjur_yearly = [info['tekjur'][0]*12,\
                     info['tekjur'][1]*12,\
                     info['tekjur'][2]*12,\
                     info['tekjur'][3]*12]
    
    eignaskerdingarmork_nedri = [4500000.0, 4500000.0, 7300000.0] #fyrir: [einstakling, einstætt foreldri, hjón]
    eignaskerdingarmork_efri = [7200000.0, 7200000.0, 11680000.0] #fyrir: [einstakling, einstætt foreldri, hjón]

    stofn_list = [hamark_vaxtagjalda[info['hjuskaparstada']-1], 
                  0.07*info['husnaedislan'], 
                  info['vaxtagjold']]
    stofn =  min(float(s) for s in stofn_list)
    
    tekjuskerding = (0.085 * sum(tekjur_yearly))
    baetur = stofn - tekjuskerding
    if baetur < 0:
        baetur = 0
    
    if info['eignir'] >= eignaskerdingarmork_efri[info['hjuskaparstada']-1]:
        eignaskerding = baetur
        baetur = 0
        
    elif info['eignir'] >= eignaskerdingarmork_nedri[info['hjuskaparstada']-1]:
        eignaskerding =\
            baetur *\
            ((info['eignir'] - eignaskerdingarmork_nedri[info['hjuskaparstada']-1]) /
            (eignaskerdingarmork_efri[info['hjuskaparstada']-1] - 
             eignaskerdingarmork_nedri[info['hjuskaparstada']-1]))
        baetur -= eignaskerding
    
    else:
        eignaskerding = 0
            
    if baetur > hamark_bota[info['hjuskaparstada']-1]:
        baetur = hamark_bota[info['hjuskaparstada']-1]

    if hjuskaparstada == 3:
        baetur = baetur/24 #bætur deilast jafnt milli hjóna
    else:
        baetur = baetur/12
    tekjuskerding = tekjuskerding/12
    eignaskerding = (eignaskerding/12)
    
    if baetur < 0:
        baetur = 0
    if tekjuskerding < 0:
        tekjuskerding = 0
    if eignaskerding < 0:
        eignaskerding = 0
            
    return baetur, tekjuskerding, eignaskerding

In [108]:
def husnaedisbaetur_func(info):

    grunnfjarhaedir = [372000/12, 492000/12, 576000/12, 624000/12]
    fritekjumark = [3373000/12, 4461064/12, 5222710/12, 5657936/12]
    eignaskerdingarmork_nedri = 6500000.0
    eignaskerdingarmork_efri = 10400000.0
    
    stofn = grunnfjarhaedir[(info['fjoldi_heimilismanna']-1)]
    
    if info['heimilistekjur'] <= fritekjumark[info['fjoldi_heimilismanna']-1]:
        baetur = stofn
        tekjuskerding = 0
    else:
        tekjuskerding = 0.09 *\
        (info['heimilistekjur'] - fritekjumark[info['fjoldi_heimilismanna']-1])
        baetur = stofn - tekjuskerding
    
    if info['heimiliseignir'] >= eignaskerdingarmork_efri:
        eignaskerding = baetur
        baetur = 0
    elif info['heimiliseignir'] >= eignaskerdingarmork_nedri:
        eignaskerding = baetur *\
            ((info['heimiliseignir'] - eignaskerdingarmork_nedri) /
             (eignaskerdingarmork_efri - eignaskerdingarmork_nedri))
        baetur -= eignaskerding
    else:
        eignaskerding = 0
    
    if baetur > 0.75*info['husnaediskostnadur']:
        baetur = 0.75*info['husnaediskostnadur']
    if baetur < 0:
        baetur = 0
    if tekjuskerding < 0:
        tekjuskerding = 0
    if eignaskerding < 0:
        eignaskerding = 0
            
    return baetur, tekjuskerding, eignaskerding

In [85]:
def barnabaetur_func(info):
    tekjur_yearly = [info['tekjur'][0]*12, 
                     info['tekjur'][1]*12, 
                     info['tekjur'][2]*12, 
                     info['tekjur'][3]*12]
    
    fj_barna = info['fjoldi_barna']
    fj_barna_undir_7 = info['fjoldi_barna_undir_7']
    
    # SKERÐINGARHLUFÖLL
    skerdingarhlutfoll = [0.04, 0.06, 0.08] # [1 barn, 2 börn, 3 börn eða fleiri]
    skerdingarhlutf_vidbot = 0.04
    
    # FJÁRHÆÐIR 2017
    fjarhaedir_fyrsta_barn = [342939, 205834] # [einstætt foreldri, hjón/sambúðarfólk]
    fjarhaedir_umfram_born = [351787, 245087] # [einstætt foreldri, hjón/sambúðarfólk]
    vidbotarfjarhaed_hvert_barn_undir_7 = 122879
    
    # SKERÐINGARMÖRK
    skerdingarmork = [2700000, 5400000] # [einstætt foreldri, hjón/sambúðarfólk]
    
    # ÚTREIKNINGUR ÓSKERTRAR BÓTAFJÁRHÆÐAR
    stofn_til_skerdingar = sum(tekjur_yearly) - skerdingarmork[info['hjuskaparstada']-2]
    if stofn_til_skerdingar < 0:
        stofn_til_skerdingar = 0
        
    almennar_barnabaetur =\
        fjarhaedir_fyrsta_barn[info['hjuskaparstada']-2] +\
        ((fj_barna-1) * fjarhaedir_umfram_born[info['hjuskaparstada']-2]) 
        
    vidbotarbaetur = (fj_barna_undir_7 * vidbotarfjarhaed_hvert_barn_undir_7)
        
    skerding = (skerdingarhlutfoll[fj_barna-1] * stofn_til_skerdingar)
    if skerding > almennar_barnabaetur:
        skerding = almennar_barnabaetur
    skerding_vidbotar = ((fj_barna_undir_7*skerdingarhlutf_vidbot) * stofn_til_skerdingar)
    if skerding_vidbotar > vidbotarbaetur:
        skerding_vidbotar = vidbotarbaetur
    
    baetur = almennar_barnabaetur + vidbotarbaetur - (skerding + skerding_vidbotar)
    if baetur < 0:
        baetur = 0
    
    if hjuskaparstada == 2:
        return baetur/12
    else:
        return baetur/24 #bætur deilast jafnt milli hjóna


In [44]:
def print_dict(person_dict):
    print ""
    print ""
    print "==========LAUN============"
    print "Laun fyrir skatt: {:,}".format(person_dict['tekjur'][0])
    print " - staðgreiðsla: {:,}".format(person_dict['stadgreidsla'])
    print " - greitt i lifeyrissparnad: {:,}".format(person_dict['lifeyrisgreidsla'])
    print "Útborguð laun: {:,}".format(person_dict['tekjur_e_skatt'])
    print ""
    print "====HÚSNÆÐISSTUÐNINGUR============"
    print "Húsnæðisstuðningur á mánuði: {:,}".format(person_dict['husnaedisstudningur'])
    print ""
    print "====Barnabætur========="
    print "Barnabætur á mánuði: {:,}".format(person_dict['barnabaetur'])
    print ""
    print "=====FRÁDRÁTTUR========"
    print "Útvarpsgjald: {:,}".format(person_dict['utvarpsgjald'])
    print ""
    print ""
    print "====NIÐURSTAÐA========"
    print "Pjénge í vasann í hverjum mánuði: {:,}"\
        .format(person_dict['pjeng_i_vasa'])
    print "======================"

In [ ]:
def pen_i_vasa(tekjur):
    tekjur_e_skatt, stadgreidsla, lifeyrisgreidsla, sereignarlifeyrisgr = stadgreidsla_func(tekjur[0])
    husnaedisstudningur, husn_eignaskerding, husn_tekjuskerding =\
        husnaedisstudningur_func(tekjur, hjuskaparstada)
    if hjuskaparstada != 1:
        barnabaetur = barnabaetur_func(hjuskaparstada, tekjur)
    else: barnabaetur = 0
    
    if tekjur[0] >= 1678001/12:
        utvarpsgjald = 16800/12
    else: utvarpsgjald = 0
    
    #print_all(tekjur, tekjur_e_skatt, stadgreidsla, lifeyrisgreidsla,\
    #          sereignarlifeyrisgr, husnaedisstudningur, barnabaetur, utvarpsgjald)
    netto = tekjur_e_skatt + husnaedisstudningur + barnabaetur - utvarpsgjald
    
    nidurstodur = {'tekjur': tekjur,\
                   'tekjur_e_skatt': tekjur_e_skatt,\
                   'stadgreidsla': stadgreidsla,\
                   'lifeyrisgreidsla': (lifeyrisgreidsla+sereignarlifeyrisgr),\
                   'husnaedisstudningur': husnaedisstudningur,\
                   'barnabaetur': barnabaetur,\
                   'utvarpsgjald': utvarpsgjald,\
                   'pjeng_i_vasa': netto}
    
    return nidurstodur  

# Reiknireikn 

In [13]:
kelson = islendingur_input()

1: einhleypur, 2: einst. foreldri, 3: giftur/sambud 1
Laun fyrir skatt: 400000
Fjarmagnstekjur: 20000
	Iðgjald í lífeyrissjóð?
1: 0%, 2: 4% 2
	Iðgjald í séreign?
0: 0%, 1: 1%, 2: 2%, 3: 3%, 4: 4% 4
Hver er husnæðisstaða þín? Býrðu í eigin húsnæði, leiguhúsnæði eða hvorugt?
1: eigin, 2: leigu, 3: hvorugt 2
fjoldi heimilismanna ad ther medtoldum: 1
samanlagdar eignir allra heimilismanna: 1000000
manadarlegur husnaediskostnadur: 240000


In [14]:
kelson

{'allar_baetur': [18497.47, 0],
 'allar_tekjur': [[400000, 20000, 0, 0], 284967.8],
 'fradrattur': [1400, 83032.20000000001, 16000.0],
 'pjeng_i_vasa': 302065.27}

In [50]:
sextuttuguthus_a_manudi = islendingur_input()

1: einhleypur, 2: einst. foreldri, 3: giftur/sambud 2
Laun fyrir skatt: 600000
Fjarmagnstekjur: 20000
	Iðgjald í lífeyrissjóð?
1: 0%, 2: 4% 2
	Iðgjald í séreign?
0: 0%, 1: 1%, 2: 2%, 3: 3%, 4: 4% 4
Hver er husnæðisstaða þín? Býrðu í eigin húsnæði, leiguhúsnæði eða hvorugt?
1: eigin, 2: leigu, 3: hvorugt 2
fjoldi heimilismanna ad ther medtoldum: 2
samanlagdar manadartekjur allra heimilismanna: 620000
samanlagdar eignir allra heimilismanna: 1000000
manadarlegur husnaediskostnadur: 240000
Fjöldi barna á heimli: 
1
 - þar af yngri en 7 ára: 
1


In [37]:
sextuttuguthus_a_manudi

{'barnabaetur': 12778.25,
 'husnaedisstudningur': 18657.95,
 'lifeyrisgreidsla': 24000.0,
 'pjeng_i_vasa': 431034.39999999997,
 'stadgreidsla': 151001.80000000002,
 'tekjur': [600000, 20000, 0, 0],
 'tekjur_e_skatt': 400998.19999999995,
 'utvarpsgjald': 1400}

In [51]:
print_dict(sextuttuguthus_a_manudi)



==========LAUN============
Laun fyrir skatt: 600,000
 - staðgreiðsla: 151,001.8
 - greitt i lifeyrissparnad: 48,000.0
Útborguð laun: 400,998.2

====HÚSNÆÐISSTUÐNINGUR============
Húsnæðisstuðningur á mánuði: 18,657.95

====Barnabætur=========
Barnabætur á mánuði: 12,778.25

=====FRÁDRÁTTUR========
Útvarpsgjald: 1,400


====NIÐURSTAÐA========
Pjénge í vasann í hverjum mánuði: 431,034.4


In [18]:
print_dict(kelson)



==========LAUN============
Tekjur fyrir skatt: 400,000
 - staðgreiðsla: 83,032.2
 - greitt i lifeyrissparnad: 16,000.0
Útborguð laun: 284,967.8

====HÚSNÆÐISSTUÐNINGUR============
Húsnæðisstuðningur á mánuði: 18,497.47

====Barnabætur=========
Barnabætur á mánuði: 0

=====FRÁDRÁTTUR========
Útvarpsgjald: 1,400


====NIÐURSTAÐA========
Pjénge í vasann í hverjum mánuði: 302,065.27
